In [1]:
import pyarabic.araby as araby
import pyarabic.number as number
import string
import re
import pickle
from collections import Counter
import torch
from torch.utils.data.dataset import Dataset, TensorDataset
from torch.utils.data.dataloader import DataLoader

In [28]:
# letter_to_id = {}
# id_to_letter = {}
# word_to_id = {}
# id_to_word = {}
# diacritic_to_id = {}
# id_to_diacritic = {}
# word_count = Counter()

# id_letter = 0
# id_word = 0
# id_diacritic = 1

# diacritic_to_id["space"] = 0
# id_to_diacritic[0] = "space"

with open('Dataset_with_shaddah/val_cleaned_withshadda_215.txt','r',encoding='utf-8') as file:
    for line in file:
#         letters, diacritics =araby.separate(line)
#         index = 0
#         for index,letter in enumerate(letters):
#             if (letter == '\n') or (letter == '\u200f'):
#                 continue
                
#             if letter == 'ّ':
#                 if diacritics[index] != 'ـ' :
#                     shaddad_with_diac = letter+diacritics[index]
#                     if shaddad_with_diac not in diacritic_to_id:
#                         diacritic_to_id[shaddad_with_diac] = id_diacritic
#                         id_to_diacritic[id_diacritic] = shaddad_with_diac
#                         id_diacritic += 1
#                 else:
#                     if letter not in diacritic_to_id:
#                         diacritic_to_id[letter] = id_diacritic
#                         id_to_diacritic[id_diacritic] = letter
#                         id_diacritic += 1
#             else:
#                 if letter not in letter_to_id:
#                     letter_to_id[letter] = id_letter
#                     id_to_letter[id_letter] = letter
#                     id_letter += 1
                    
#                 diacritic = diacritics[index]
#                 if diacritic not in diacritic_to_id:
#                     diacritic_to_id[diacritic] = id_diacritic
#                     id_to_diacritic[id_diacritic] = diacritic
#                     id_diacritic += 1
                    
        words = araby.tokenize(line)
        for word in words:
            if word == '\n':
                continue
            word_count[word] += 1
            
            if word not in word_to_id:
                word_to_id[word] = id_word
                id_to_word[id_word] = word
                id_word +=1
                

In [29]:
len(id_to_word)

86779

In [ ]:
with open('Dataset_with_shaddah/letter_to_id.pickle', 'wb') as file:
    pickle.dump(letter_to_id, file)
    
with open('Dataset_with_shaddah/id_to_letter.pickle', 'wb') as file:
    pickle.dump(id_to_letter, file)

In [30]:
with open('Dataset_with_shaddah/word_to_id_withdiacritics.pickle', 'wb') as file:
    pickle.dump(word_to_id, file)
    
with open('Dataset_with_shaddah/id_to_word_withdiacritics.pickle', 'wb') as file:
    pickle.dump(id_to_word, file)

In [ ]:
with open('Dataset_with_shaddah/diacritic_to_id.pickle', 'wb') as file:
    pickle.dump(diacritic_to_id, file)
    
with open('Dataset_with_shaddah/id_to_diacritic.pickle', 'wb') as file:
    pickle.dump(id_to_diacritic, file)

In [31]:
with open('Dataset_with_shaddah/word_count_withdiacritics.pickle', 'wb') as file:
    pickle.dump(word_count, file)

In [ ]:
with open('Dataset_with_shaddah/id_to_word.pickle', 'rb') as file:
    with_shaddah = pickle.load(file)
len(with_shaddah)

In [ ]:
with open('Dataset/id_to_word.pickle', 'rb') as file:
    no_shaddah = pickle.load(file)
len(no_shaddah)

In [2]:
class DiacriticDatasetShaddah(Dataset):
    def __init__(self,dataset_path,letter_to_id_path,id_to_letter_path,diacritic_to_id_path,id_to_diacritic_path,word_to_id_path,id_to_word_path):
        
        self.file = dataset_path
        
        letter_to_id_file= open(letter_to_id_path, 'rb')
        self.letter_to_id = pickle.load(letter_to_id_file)
        letter_to_id_file.close()
        
        id_to_letter_file = open(id_to_letter_path, 'rb')
        self.id_to_letter = pickle.load(id_to_letter_file)
        id_to_letter_file.close()
        
        diacritic_to_id_file= open(diacritic_to_id_path, 'rb')
        self.diacritic_to_id = pickle.load(diacritic_to_id_file)
        diacritic_to_id_file.close()
        
        id_to_diacritic_file = open(id_to_diacritic_path, 'rb')
        self.id_to_diacritic = pickle.load(id_to_diacritic_file)
        id_to_diacritic_file.close()
        
        word_to_id_file= open(word_to_id_path, 'rb')
        self.word_to_id = pickle.load(word_to_id_file)
        word_to_id_file.close()
        
        id_to_word_file = open(id_to_word_path, 'rb')
        self.id_to_word = pickle.load(id_to_word_file)
        id_to_word_file.close()
        
        self.data = self.prepare_dataset()
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
        return self.data[index]
    
    def prepare_dataset(self):
        data = {}
        counter = 0
        with open(self.file,encoding='utf-8') as file:
            for line in file:
                letter_ids = []
                diacritic_ids = []
                word_ids = []
                letters, diacritics =araby.separate(line)
                letters = letters[0:-1]
                words = araby.tokenize(line)[0:-1]
                diacritics = diacritics[0:-1]
                index = 0
                for letter in letters:
                    if (letter == '\n') or (letter == '\u200f'):
                        continue
                    if (letter =='ّ'):
                        if diacritics[index] != 'ـ':
                            diacritic_ids[-1] = self.diacritic_to_id[letter+diacritics[index]]
                            
                        else:
                            diacritic_ids[-1] = self.diacritic_to_id[letter]
                        
                            
                    else:
                        letter_ids.append(self.letter_to_id[letter])
                        if letter == " ":
                            diacritic_ids.append(self.diacritic_to_id['space'])
                            
                        else:
                            diacritic_ids.append(self.diacritic_to_id[diacritics[index]])
                            
                    index += 1
                    
                for word in words:
                    word_ids.append(self.word_to_id[word])

                instance = (torch.tensor(letter_ids,dtype=torch.long,requires_grad=False),
                           torch.tensor(diacritic_ids,dtype=torch.long,requires_grad=False),
                           torch.tensor(word_ids,dtype=torch.long,requires_grad=False))
                data[counter] = instance
                counter += 1
        return data

In [3]:
train = DiacriticDatasetShaddah('Dataset_with_shaddah/train_cleaned_withshadda_215.txt','Dataset_with_shaddah/letter_to_id.pickle','Dataset_with_shaddah/id_to_letter.pickle','Dataset_with_shaddah/diacritic_to_id.pickle','Dataset_with_shaddah/id_to_diacritic.pickle','Dataset_with_shaddah/word_to_id.pickle','Dataset_with_shaddah/id_to_word.pickle')

In [6]:
train_dataloader = DataLoader(train)

In [12]:
for batch_idx, input_data in enumerate(train_dataloader):
    letters = input_data[0]
    labels = input_data[1]
    if letters.size(1) != labels.size(1):
        print("Error")

In [4]:
bug = 'اللَّهُمَّ إِنَّكَ تَقِي الْكَرِيهَةَ وَ تُعْطِي الْحَسَنَةَ وَ تَفْعَلُ مَا تُرِيدُ وَ أَنْتَ عَلَى كُلِّ شَيْءٍ قَدِيرٌ'
letters, diacritics =araby.separate(bug)
print(len(letters),len(diacritics))

76 76


In [5]:
count = 0
total = 0
with open('Dataset_with_shaddah/test_cleaned_withshadda_215.txt','r',encoding='utf-8') as file:
    for line in file:
        words = araby.tokenize(line)
        for word in words:
            if word == '\n':
                continue
            
            if araby.strip_shadda(word) != word:
                count += 1
        total += len(words)
        
print(count/total)

0.17616249361267247


In [10]:
with open('Dataset_with_shaddah/test_cleaned_withshadda_215.txt','r',encoding='utf-8') as file:
    test, split, val = file.read().partition("وَقَوْلُهُ وَلَوْ حَلَفَ لَا يَجْلِسُ عَلَى سَرِيرٍ ظَاهِرٌ مِمَّا تَقَدَّمَ")
with open('Dataset_with_shaddah/test1_cleaned_withshadda_215.txt', 'w') as f:
    f.write(test)
with open('Dataset_with_shaddah/val_cleaned_withshadda_215.txt', "w") as f:
    f.write(split)
    f.write(val)

In [4]:
with open('Dataset_with_shaddah/id_to_word.pickle', 'rb') as f:
    data = pickle.load(f)
data

{0: 'ولو',
 1: 'جمع',
 2: 'ثم',
 3: 'علم',
 4: 'ترك',
 5: 'ركن',
 6: 'من',
 7: 'الأولى',
 8: 'بطلتا',
 9: 'ويعيدهما',
 10: 'جامعا',
 11: 'أو',
 12: 'الثانية',
 13: 'فإن',
 14: 'لم',
 15: 'يطل',
 16: 'تدارك',
 17: 'وإلا',
 18: 'فباطلة',
 19: 'ولا',
 20: 'جهل',
 21: 'أعادهما',
 22: 'لوقتيهما',
 23: 'قال',
 24: 'أبو',
 25: 'زيد',
 26: 'أهل',
 27: 'تهامة',
 28: 'يؤنثون',
 29: 'العضد',
 30: 'وبنو',
 31: 'تميم',
 32: 'يذكرون',
 33: 'والجمع',
 34: 'أعضد',
 35: 'وأعضاد',
 36: 'مثل',
 37: 'أفلس',
 38: 'وأقفال',
 39: 'بمنزلة',
 40: 'الذمة',
 41: 'إذا',
 42: 'دخلوا',
 43: 'قرية',
 44: 'قرى',
 45: 'الحرب',
 46: 'ظفر',
 47: 'المسلمون',
 48: 'بها',
 49: 'فهم',
 50: 'فيء',
 51: 'أجمعون',
 52: 'إلا',
 53: 'عرف',
 54: 'أنه',
 55: 'ذمي',
 56: 'لأنه',
 57: 'يبق',
 58: 'لسيده',
 59: 'معه',
 60: 'حق',
 61: 'عدة',
 62: 'على',
 63: 'أمة',
 64: 'ومدبرة',
 65: 'كان',
 66: 'يطؤها',
 67: 'لعدم',
 68: 'الفراش',
 69: 'جوهرة',
 70: 'و',
 71: 'كذا',
 72: 'موطوءة',
 73: 'بشبهة',
 74: 'كمزفوفة',
 75: 'لغير',
 76: 'بعل

In [7]:
with open('Dataset_with_shaddah/diacritic_to_id.pickle','rb') as file:
    diacritic_to_id = pickle.load(file)
    
with open('Dataset_with_shaddah/id_to_diacritic.pickle','rb') as file:
    id_to_diacritic = pickle.load(file)
   
diacritic_to_id['*'] = 16
id_to_diacritic[16] = '*'

In [9]:
with open('Dataset_with_shaddah/diacritic_to_id_memm.pickle','wb') as file:
    pickle.dump(diacritic_to_id,file)
    
with open('Dataset_with_shaddah/id_to_diacritic_memm.pickle','wb') as file:
    pickle.dump(id_to_diacritic,file)

In [12]:
with open('Dataset_with_shaddah/letter_to_id.pickle','rb') as file:
    letter_to_id = pickle.load(file)
    
with open('Dataset_with_shaddah/id_to_letter.pickle','rb') as file:
    id_to_letter = pickle.load(file)
   
letter_to_id['*'] = 37
id_to_letter[37] = '*'

In [15]:
with open('Dataset_with_shaddah/letter_to_id_memm.pickle','wb') as file:
    pickle.dump(letter_to_id,file)
    
with open('Dataset_with_shaddah/id_to_letter_memm.pickle','wb') as file:
    pickle.dump(id_to_letter,file)

In [26]:
with open('Dataset_with_shaddah/word_to_id.pickle','rb') as file:
    word_to_id = pickle.load(file)
    
with open('Dataset_with_shaddah/id_to_word.pickle','rb') as file:
    id_to_word = pickle.load(file)

word_to_id['*'] = 59481
id_to_word[59481]= '*'

In [28]:
with open('Dataset_with_shaddah/word_to_id_memm.pickle','wb') as file:
    pickle.dump(word_to_id,file)
    
with open('Dataset_with_shaddah/id_to_word_memm.pickle','wb') as file:
    pickle.dump(id_to_word,file)

In [30]:
with open('Dataset_with_shaddah/word_count.pickle','rb') as file:
    word_count = pickle.load(file)
    
word_count

Counter({'ولو': 2882,
         'جمع': 206,
         'ثم': 2335,
         'علم': 419,
         'ترك': 230,
         'ركن': 22,
         'من': 13820,
         'الأولى': 199,
         'بطلتا': 2,
         'ويعيدهما': 1,
         'جامعا': 4,
         'أو': 9294,
         'الثانية': 120,
         'فإن': 2649,
         'لم': 5469,
         'يطل': 10,
         'تدارك': 3,
         'وإلا': 824,
         'فباطلة': 6,
         'ولا': 3311,
         'جهل': 87,
         'أعادهما': 3,
         'لوقتيهما': 1,
         'قال': 5999,
         'أبو': 796,
         'زيد': 266,
         'أهل': 546,
         'تهامة': 1,
         'يؤنثون': 1,
         'العضد': 9,
         'وبنو': 2,
         'تميم': 45,
         'يذكرون': 3,
         'والجمع': 12,
         'أعضد': 1,
         'وأعضاد': 1,
         'مثل': 455,
         'أفلس': 14,
         'وأقفال': 2,
         'بمنزلة': 150,
         'الذمة': 77,
         'إذا': 2354,
         'دخلوا': 6,
         'قرية': 27,
         'قرى': 6,
         'الحرب': 126,
      